In [1]:
#数据处理部分之前的代码，加入部分故起处理的库
import paddle
from paddle.nn import Conv2D, MaxPool2D, Linear
import paddle.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from data_process import get_MNIST_dataloader
from tools import plot
train_loader,_ = get_MNIST_dataloader()


c:\Users\25588\AppData\Local\Programs\Python\Python311\Lib\site-packages\paddle\io\reader.py:433: UserWarning: DataLoader with multi-process mode is not supported on MacOs and Windows currently. Please use signle-process mode with num_workers = 0 instead
  warnings.warn(


In [2]:
#定义多层全连接神经网络
class MNIST(paddle.nn.Layer):
    def __init__ (self):
        super(MNIST, self).__init__ ()
        #定义两层全连接隐含层，输出维度是10，当前设定隐含节点数为10，可根据任务调整
        self.fc1 = Linear(in_features=784, out_features=10)
        self.fc2 = Linear(in_features=10, out_features=10)
        #定义一层全连接输出层，输出维度是1
        self.fc3 = Linear(in_features=10,out_features=1)
    #定义网络的前向计算，隐含层激活函教为sigmoid，输出层不使用激活函数
    def forward(self, inputs):
        inputs = paddle.reshape(inputs, [inputs.shape[0], 784])
        outputs1 = self.fc1(inputs)
        outputs1 = F.sigmoid(outputs1)
        outputs2 = self.fc2(outputs1)
        outputs2 = F.sigmoid(outputs2)
        outputs_final = self.fc3(outputs2)
        return outputs_final

In [3]:
model = MNIST()
params_info = paddle.summary(model, (1, 1, 28, 28))
print(params_info)

---------------------------------------------------------------------------
 Layer (type)       Input Shape          Output Shape         Param #    
   Linear-1          [[1, 784]]            [1, 10]             7,850     
   Linear-2          [[1, 10]]             [1, 10]              110      
   Linear-3          [[1, 10]]              [1, 1]              11       
Total params: 7,971
Trainable params: 7,971
Non-trainable params: 0
---------------------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.03
Estimated Total Size (MB): 0.03
---------------------------------------------------------------------------

{'total_params': 7971, 'trainable_params': 7971}


In [4]:
#网络结构部分之后的代码，保持不变
def train(model):
    model.train()
    # 使用SGD优化器，learning_rate设置为0.01
    opt = paddle.optimizer.SGD(learning_rate=0.01, parameters=model.parameters())
     # 训练5轮
    EPOCH_NUM = 10
    loss_list = []
    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            #准备数据
            images, labels = data
            images = paddle.to_tensor(images)
            Iabels = paddle.to_tensor(labels, dtype="float32")
            #前向计算的过程
            predicts = model(images)
            #计算损失，取一个批次样本损失的平均值
            loss = F.square_error_cost(predicts, labels)
            avg_loss = paddle.mean(loss)
            #每训练200批次的数据，打印下当前Loss的情况
            if batch_id % 200 == 0:
                loss_list.append(avg_loss.numpy()[0])
                print("epoch: ({), batch: [), loss is: ()".format(epoch_id, batch_id, avg_loss.numpy()))
            #后向传播，更新参数的过程
            avg_loss.backward()
            #最小化loss,更新参数
            opt.step()# 清除梯度
            opt.clear_grad()
    #保存模型参数
    paddle.save(model.state_dict(), "C:\\Users\\25588\\Desktop\\python程序设计\\shendu")
    return loss_list
model = MNIST()
loss_list = train(model)

ValueError: (InvalidArgument) The type of data we are trying to retrieve (int64) does not match the type of data (float32) currently contained in the container.
  [Hint: Expected dtype() == phi::CppTypeToDataType<T>::Type(), but received dtype():10 != phi::CppTypeToDataType<T>::Type():9.] (at ..\paddle\phi\core\dense_tensor.cc:166)


In [ ]:
#多卷积神经网络实现
class MNIST(paddle.nn.Layer):
    def __init__(self):
        super(MNIST, self).__init__ ()
       # 定义卷积层，输出特征通道out_channels设置为20，卷积核的大小kernel_size为5，卷积步长stride=1，padding=2
        self.conv1 = (Conv2D(in_channels=1, out_channels=28, kernel_size=5, stride=1, padding=2))
        #定义池化层，池化校的大小kernel_size为2，池化步长为2
        self.max_pool1 = MaxPool2D(kernel_size=2,stride=2)
        # 定义卷职层，输出特征通道out_channels设置为20，卷积核的大小kernel_size为5，卷积步长stride=1，padding=2
        self.conv2 = Conv2D(in_channels=20, out_channels=28, kernel_size=5, stride=1, padding=2)
        #开定义池化层，池化校核的大小kernel_size为2，池化步长为2
        self.max_pool2 = MaxPool2D(kernel_size=2, stride=2)
        #定义一层全连接层，输出维度是1
        self.fc = Linear(in_features=980, out_features=1)
    #定义网络前向计算过程，卷积后紧接着使用池化层，最后使用全连按层计算最终输出#卷积层激活函数使用Relu，全连越层不使用激活函数
    def forward(self, inputs):
        x = self.conv1(inputs)
        x = F.relu(x)
        x = self.max_pool1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.max_pool2(x)
        x = paddle.reshape(x, [x.shape[0, -1]])
        x = self.fc(x)
        return x

In [ ]:
model = MNIST()
params_info = paddle.summary(model,(1, 1, 28,28))
print(params_info)

In [ ]:
#网络结构部分之后的代码，保持不变
def train(model):
    model.train()
    learning_rate = 0.001
    # 使用SGD优化器，设置learning_rate
    opt = paddle.optimizer.SGD(learning_rate=learning_rate, parameters=model.paraneters())
    # 训练5轮
    EPOCH_NUM = 10
    # MNIST图像高和宽
    IMG_ROWS, IMG_COLS = 28, 28
    loss_list = []
    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            #准备数据
            images, labels = data
            images = paddle,to_tensor(images)
            labels = paddle.to_tensor(labels, dtype="float32")
            #前向计算的过程
            predicts = model(images) # [batch_size, 1]
            #计算损失，取一个批次样本损失的平均值
            loss = F.square_error_cost(predicts, labels)
            avg_loss = paddle.mean(loss)
            #每训练200批次的数据，打印下当前Loss的情况
            if batch_id % 200 == 0:
                loss_list.append(avg_loss .numpy()[0])
                print("epoch: (), batch: (), loss is: ()".format(epoch_id, batch_id, avg_loss.numpy()))
            #后向传播，更新参数的过程
            avg_loss.backward()
            # 最小化loss,更新参数
            opt.step()
            # 清除梯度
            opt.clear_grad()
    #保存模型参数
    paddle.save(model.state_dict(), 'D:/datasets/pics/projects/digitalRecognition/mnistN2.pdparams')
    return loss_list
rodel = MNIST()
loss_list_conv=train(model)

In [ ]:
def plot_two_losses(loss_list_1, loss_list_2):
    plt.figure(figsize=(10,5))
    freqs = [i for i in range(len(loss_list_1))]
    # 绘制训练损失变化曲线
    plt.plot(freqs, loss_list_1, color='#e4007f', label="Train loss1")
    plt.plot(freqs, loss_list_2, color='#f19ec2',linestyle='--', label="Train loss2")
    #绘制坐标轴和图例fontsize='large')
    plt.ylabel("loss", fontsize = 'large')
    plt.xlabel("freq", fontsize = 'large')
    plt.legend(loc='upper right', fontsize='x-large')
    plt.show()
plot two_losses(loss_list, loss_list_conv)